# Описание проекта

В добывающей компании «ГлавРосГосНефть» нужно решить, где бурить новую скважину.

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализировать возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Описание данных

- *id* — уникальный идентификатор скважины
- *f0*, *f1*, *f2* — три признака точек (неважно, что они означают, но сами признаки значимы)
- *product* — объём запасов в скважине (тыс. баррелей)

## Оглавление

1. [Загрузка и подготовка данных](#step1)
2. [Обучение и проверка модели](#step2)
3. [Подготовка к расчёту прибыли](#step3)
4. [Расчёт прибыли и рисков](#step4)
5. [Общий вывод](#step5)

<a id="step1"></a>
## 1. Загрузка и подготовка данных

In [1]:
# Подключение библиотек, необходимых модулей и алгоритмов
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState

from IPython.display import display

Определим константные значения, которые мы будем использовать в дальнейшем.

In [2]:
# количество регионов в данном исследовании
NUM_REGIONS = 3

# Параметр рандомизиции
RAND_SEED = 12345

Прочитаем данные из файлов, сохраним в соответствующие переменные и выведем первые 10 строк для каждой таблицы.

In [3]:
geo_data = []
for i in range(NUM_REGIONS):
    geo_data.append(pd.read_csv('/datasets/geo_data_' + str(i) + '.csv'))
    print('Geo_data for region {}:'.format(i))
    display(geo_data[i].head(10))
    print(60 * '-')

Geo_data for region 0:


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


------------------------------------------------------------
Geo_data for region 1:


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
5,HHckp,-3.327590,-2.205276,3.003647,84.038886
6,h5Ujo,-11.142655,-10.133399,4.002382,110.992147
7,muH9x,4.234715,-0.001354,2.004588,53.906522
8,YiRkx,13.355129,-0.332068,4.998647,134.766305
9,jG6Gi,1.069227,-11.025667,4.997844,137.945408


------------------------------------------------------------
Geo_data for region 2:


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
5,LzZXx,-0.758092,0.710691,2.585887,90.222465
6,WBHRv,-0.574891,0.317727,1.773745,45.641478
7,XO8fn,-1.906649,-2.458350,-0.177097,72.480640
8,ybmQ5,1.776292,-0.279356,3.004156,106.616832
9,OilcN,-1.214452,-0.439314,5.922514,52.954532


------------------------------------------------------------


Теперь, когда мы сохранили данные, посмотрим на общую информацию для каждой таблицы

In [4]:
for i in range(NUM_REGIONS):
    print('Region {}:'.format(i))
    print(geo_data[i].columns)
    print(30 * '-')
    display(geo_data[i].info())
    print(60 * '/')

Region 0:
Index(['id', 'f0', 'f1', 'f2', 'product'], dtype='object')
------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

////////////////////////////////////////////////////////////
Region 1:
Index(['id', 'f0', 'f1', 'f2', 'product'], dtype='object')
------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

////////////////////////////////////////////////////////////
Region 2:
Index(['id', 'f0', 'f1', 'f2', 'product'], dtype='object')
------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

////////////////////////////////////////////////////////////


### Вывод

На данном шаге, мы загрузили и ознакомились с данными. Проверили названия столбцов на наличие лишних пробелов, нелатинских и заглавных символов. Все названия корректны, изменения не требуется. Также была проверена общая информация по таблицам - типы данных определены правильно, пропусков не наблюдается. Можно переходить к работе с моделью.

<a id="step2"></a>
## 2. Обучение и проверка модели

На данном этапе нам необходимо обучить модель и проверить ее работу. Для этих целей нам нужно разделить наши данные на обучающую и валидационную выборки. Перед этим необходимо выделить целевой признак. В нашем случае им является `product` - объем запасов сырья в скважине. Предсказывать мы будем по трем признакам `f0`, `f1`, `f2`. Исходя из условий задачи, уникальный идентификатор скважины нам здесь не нужен, поэтому мы его удалим из признаков.

In [5]:
# Подготовка признаков
features = []
target = []
for i in range(NUM_REGIONS):
    features.append(geo_data[i].drop(['id', 'product'], axis=1))
    target.append(geo_data[i]['product'])

In [6]:
# Разделим на выборки
features_train = []
target_train = []

features_valid = []
target_valid = []

for i in range(NUM_REGIONS):
    cur_features_train, cur_features_valid, cur_target_train, cur_target_valid = train_test_split(
            features[i], target[i], test_size=0.25, random_state=RAND_SEED)
    
    features_train.append(cur_features_train)
    features_valid.append(cur_features_valid)
    target_train.append(cur_target_train)
    target_valid.append(cur_target_valid)

In [7]:
features_train[0].head()

,f0,f1,f2
27212,0.022450,0.951034,2.197333
7866,1.766731,0.007835,6.436602
62041,0.724514,0.666063,1.840177
70185,-1.104181,0.255268,2.026156
82230,-0.635263,0.747990,6.643327


Далее для каждого региона обучим модель и сохраним предсказания для валидационной выборке.

In [8]:
predicted_valid = []
for i in range(NUM_REGIONS):
    model = LinearRegression()
    model.fit(features_train[i], target_train[i])
    cur_predicted_valid = model.predict(features_valid[i])
    cur_predicted_valid = pd.Series(cur_predicted_valid, index=target_valid[i].index)
    predicted_valid.append(cur_predicted_valid)

In [9]:
# Выведем получившиеся результаты
for i in range(NUM_REGIONS):
    print('Region {}:'.format(i))
    print('predicted_valid:')
    display(predicted_valid[i][0:5])
    print(30 * '-')
    print('target_valid:')
    display(target_valid[i].head())
    print(60 * '/')

Region 0:
predicted_valid:


71751    95.894952
80493    77.572583
2655     77.892640
53233    90.175134
91141    70.510088
dtype: float64

------------------------------
target_valid:


71751     10.038645
80493    114.551489
2655     132.603635
53233    169.072125
91141    122.325180
Name: product, dtype: float64

////////////////////////////////////////////////////////////
Region 1:
predicted_valid:


...

////////////////////////////////////////////////////////////
Region 2:
predicted_valid:


71751     93.599633
80493     75.105159
2655      90.066809
53233    105.162375
91141    115.303310
dtype: float64

------------------------------
target_valid:


71751     61.212375
80493     41.850118
2655      57.776581
53233    100.053761
91141    109.897122
Name: product, dtype: float64

////////////////////////////////////////////////////////////


Для каждой региона посчитаем средний запас предсказанного сырья и RMSE полученной модели.

In [10]:
mean_predicted_valid = []
rmse_model = []
for i in range(NUM_REGIONS):
    # посчитаем среднее значение предсказанного сырья
    mean_predicted_valid.append(predicted_valid[i].mean())
    print('mean_predicted_valid[{}] = {}'.format(i, mean_predicted_valid[i]))
    
    # посчитаем значение RMSE (квадратный корень из средней квадратичной ошибки) через вычисление MSE
    mse = mean_squared_error(target_valid[i], predicted_valid[i])
    rmse_model.append(mse ** 0.5)
    print('rmse_model[{}] = {}'.format(i, rmse_model[i]))
    
    print(60 * '-')

mean_predicted_valid[0] = 92.59256778438038
rmse_model[0] = 37.5794217150813
------------------------------------------------------------
mean_predicted_valid[1] = 68.728546895446
rmse_model[1] = 0.893099286775616
------------------------------------------------------------
mean_predicted_valid[2] = 94.96504596800489
rmse_model[2] = 40.02970873393434
------------------------------------------------------------


### Вывод

На данном этапе мы выделили целевой признак, котроый мы хотим научиться предсказывать, а также значимые признаки для такого предсказания. Для каждого региона разделили выборки на обучающую и валидационную. Обучили каждую из моделей по алгоритму линейной регресии и сохранили предсказания. Далее посчитали среднее значения этих предсказаний и значение RMSE для каждой модели. Для линейной регресии, чем меньше средняя квадратичная ошибка тем лучше. Как видно из полученных результатов, самой качественной получилась модель для региона 1, значение RMSE для которой равно 0.89. Для остальных моделей видно, что RMSE уже больше по значению, следственно и правильный ответ также больше отличается от предсказания.

<a id="step3"></a>
## 3. Подготовка к расчёту прибыли

На данном этапе подготовим данные для расчета прибыли. Сохраним ключевые значения в отдельных переменных.

In [11]:
# количество исследуемых точек при разведке региона
NUM_FIELD_DEVELOP = 500
# количество лучших точек для разработки
NUM_TOP_FIELD_DEVELOP = 200
# бюджет на разработку скажин в регионе (10 млрд руб = 10 000 000 000 руб = 10 000 000 тыс. руб = 10 * 10**6 тыс. руб)
BUDGET_FOR_REGION = 10 * 10**6
# доход с каждой единицы продукта (450 тыс. руб, поскольку объём указан в тысячах баррелей)
INCOME_PER_PRODUCT_UNIT = 450
# граница вероятности убытков
PROB_LOSS_BOUND = 0.025

Необходимо рассчитать достаточный объём сырья для безубыточной разработки новой скважины. В регионе у нас разрабатываются 200 скважин. Чтобы каждая скважина была безубыточна, нужно чтобы доход со всего сырья был равен бюджету на разработку скажин в регионе.<br>
В итоге получем:<br>
`количество скважин для разработки` * (`доход с каждой единицы продукта` * `объем сырья`) == `бюджет на разработку скажин`<br>
200 * (450 * x) == 10 000 000

In [12]:
sufficient_raw_amount = (BUDGET_FOR_REGION / NUM_TOP_FIELD_DEVELOP) / INCOME_PER_PRODUCT_UNIT
print('sufficient_raw_amount =', sufficient_raw_amount)

sufficient_raw_amount = 111.11111111111111


In [13]:
mean_target_valid = []

# посчитаем средний запас сырья в каждом регионе
for i in range(NUM_REGIONS):
    mean_target_valid.append(target_valid[i].mean())
    print('mean_target_valid[{}] = {}'.format(i, mean_target_valid[i]))

mean_target_valid[0] = 92.07859674082927
mean_target_valid[1] = 68.72313602435997
mean_target_valid[2] = 94.88423280885438


### Вывод

На данном этапе мы выполняли теоретический расчет достаточного объема сырья для безубыточной разработки новой скажины. В итоге получилось, что в среднем должно добываться порядка 111.11 тыс. баррелей. Для наших регионов средние значения сырья составляют 92.08, 68.72, 94.88. Видно, что в первом и третьем регионах данный показатель ближе минимальному теоритическому. Во втором регионе, вероятность найти скважины с нужным количеством проблемотичнее - среднее меньше практически на 25 пунктов.

<a id="step4"></a>
## 4. Расчёт прибыли и рисков 

Сейчас напишем функцию для расчета прибыли по выбранным скважинам и предсказаниями модели. Для этого выберем скважины с максимальными значениями предсказаний, просуммируем целевые значения объёма сырья, соответствующие этим предсказаниям и рассчитаем прибыль для полученного объёма сырья.

In [14]:
def calc_profit(target, predictions):
    sorted_predictions = predictions.sort_values(ascending=False)
    selected = target[sorted_predictions.index][:NUM_TOP_FIELD_DEVELOP]
    revenue = selected.sum() * INCOME_PER_PRODUCT_UNIT
    profit = revenue - BUDGET_FOR_REGION
    return profit

Проверим работу функции для наших регионов.

In [15]:
profit_region = []
for i in range(NUM_REGIONS):
    profit_region.append(calc_profit(target_valid[i], predicted_valid[i]))
    print('Predicted profit for region {}: {}'.format(i, profit_region[i]))

Predicted profit for region 0: 3320826.0431398526
Predicted profit for region 1: 2415086.6966815125
Predicted profit for region 2: 2710349.9635998327


Функцию мы написали. Теперь перейдем к подсчету рисков и прибыли для каждого региона с помощью техники Bootstrap.

Чтобы всегда получать разные выборки при Bootstrap, необходимо использовать объект RandomState.

In [16]:
state = np.random.RandomState(12345)

In [17]:
reg_mean_profits = []
reg_risk_loss = []

# Рассмотрим для каждого региона
for cur_reg in range(NUM_REGIONS):
    profits = []
    # по 1000 выборок в регионе
    for i in range(1000):
        # для разведки региона исследуем 500 точек
        target_valid_subsample = target_valid[cur_reg].sample(n=NUM_FIELD_DEVELOP, replace=True, random_state=state)
        pred_valid_subsample = predicted_valid[cur_reg][target_valid_subsample.index]
        
        # посчитаем прибыли для всех выборок
        cur_profit = calc_profit(target_valid_subsample, pred_valid_subsample)
        profits.append(cur_profit)
    
    # перведем в тип данных Series
    profits = pd.Series(profits)
    
    # посчитаем среднюю прибыль
    reg_mean_profits.append(profits.mean())
    print('Mean profit for region {}: {}'.format(cur_reg, reg_mean_profits[cur_reg]))
    
    # найдем 95% доверительный интервал
    lower = profits.quantile(0.025)
    upper = profits.quantile(0.975)
    print('Lower bound of confidence interval: ', lower)
    print('Upeer bound of confidence interval: ', upper)
    
    # посчитаем риски убытков для каждого региона
    num_neg_values = profits.agg(lambda x: sum(x < 0))
    risk_loss = num_neg_values / len(profits)
    reg_risk_loss.append(risk_loss)
    print('Risk of loss: ', risk_loss)
    
    print('-' * 60)

Mean profit for region 0: 425938.5269105924
Lower bound of confidence interval:  -102090.09483793723
Upeer bound of confidence interval:  947976.3533583689
Risk of loss:  0.06
------------------------------------------------------------
Mean profit for region 1: 518259.493697325
Lower bound of confidence interval:  128123.23143308601
Upeer bound of confidence interval:  953612.9820669085
Risk of loss:  0.003
------------------------------------------------------------
Mean profit for region 2: 420194.00534405006
Lower bound of confidence interval:  -115852.60916001163
Upeer bound of confidence interval:  989629.9398445741
Risk of loss:  0.062
------------------------------------------------------------


Теперь необходимо выбрать регион, в котором вероятность убытков менее 2.5%.

In [18]:
appropriate_regs = []
for i in range(NUM_REGIONS):
    if reg_risk_loss[i] < PROB_LOSS_BOUND:
        appropriate_regs.append(i)

print(appropriate_regs)

[1]


### Вывод

На данном шаге определяли оптимально подходящий регион для разработки скважин. Для этого мы сначала написали функцию, которая по выбранным скважинам и предсказаниям моедли определяла прибыль с этих скважин. Далее с помощью техники Bootstrap на 1000 выборках определяли среднее значение прибыли для каждого региона, 95% доверительный интервал для нее, а также риски убытков.<br>
В итоге нам необходимо было определить регион для разработки скважин. Для этого необходимым условием являлось, что вероятность убытков должна быть менее 2.5%. По нашим расчетам, единственным регионом с таким показателем является регион номер 2. Вероятность убытков здесь составляет 0,3%. Среднее значение сырья для него составляет всего 68.72 тыс баррелей. Что касается предсказаний, то модель для этого региона предсказывает с высокой точностью, величина RMSE составляет 0.89. Предсказания не будут сильно разнится с фактическими значениями, а значит модели можно доверять. Среднее значение предсказываемой прибыли по нему составляет 518259.49 тыс барелей. что больше чем у остальных регионов. Что касается теоретически рассчитанного значения достаточного объема сырья для безубыточной разработки новой скажины (111.11 тыс баррелей), и того, что для этого региона среднее щначение объема сырья менее всех, то это лишь означает что в этом регионе есть множество скважин как с маленьким объемом сырья. Но засчет того, что в это же время есть скважины с большим объемом сырья и точностью предсказания модели, это нивелирует данный факт.<br>
С регионами 1 и 3, дело обстоит наоборот - в среднем объема сырья здесь больше, но модель предсказывает хуже. Это приводит к тому, что вероятность убытков достигает неприемлемого уровня. Также меньшее среднее значение прибыли в этих регионах также играет негативную роль в выборе их для разработки.

<a id="step5"></a>
## 5. Общий вывод

В данном проекте мы разрабатывали модель для выбора региона с максимальным прибылью от добычи нефти. Предсказания мы строили на алгоритме линейной регресии, а в качестве параметров брали характеристики для каждой скважины. Рассмотрели, насколько точно предсказывают наши модели. Посчитали теоретическое значение достаточного объема сырья для безубыточной разработки новой скажины. Далее с помощью техники Bootstrap определили регион с минимальным значением вероятности убытков. Таким оказался регион 2.